In [1]:
%matplotlib inline
%reload_ext autoreload
%autoreload 2
%config InlineBackend.figure_format = 'retina'

In [14]:
import logging

FORMAT = '%(asctime)s %(message)s'

logging.basicConfig(level='INFO', format=FORMAT, datefmt='%d-%b-%y %H:%M:%S')

In [2]:
epoch_key = ("Jaq", 3, 2)

In [3]:
from src.load_data import load_data

data = load_data(epoch_key)

Failed to load file: /home/edeno/Documents/Github/pose_analysis/src/../Raw-Data/Jaq/JaqDIO03.mat
No DIO file found, using distance from well to segment trials
Failed to load file: /home/edeno/Documents/Github/pose_analysis/src/../Raw-Data/Jaq/JaqDIO03.mat
No DIO file found, inferring correct inbound/outbound from task rules
Failed to load file: /home/edeno/Documents/Github/pose_analysis/src/../Raw-Data/Jaq/Jaqmarks03.mat
Failed to load file: /home/edeno/Documents/Github/pose_analysis/src/../Raw-Data/Jaq/Jaqmarks03.mat
Failed to load file: /home/edeno/Documents/Github/pose_analysis/src/../Raw-Data/Jaq/Jaqmarks03.mat
Failed to load file: /home/edeno/Documents/Github/pose_analysis/src/../Raw-Data/Jaq/Jaqmarks03.mat
Failed to load file: /home/edeno/Documents/Github/pose_analysis/src/../Raw-Data/Jaq/Jaqmarks03.mat
Failed to load file: /home/edeno/Documents/Github/pose_analysis/src/../Raw-Data/Jaq/Jaqmarks03.mat
Failed to load file: /home/edeno/Documents/Github/pose_analysis/src/../Raw-Data/

In [ ]:
data["spikes"].shape

In [9]:
from dask.distributed import Client

client = Client(n_workers=16, threads_per_worker=4)
client

Client Scheduler: tcp://127.0.0.1:43277 Dashboard: http://127.0.0.1:39435/status,Cluster Workers: 16 Cores: 64 Memory: 1.62 TB


In [ ]:
from replay_trajectory_classification import SortedSpikesClassifier

from src.parameters import (discrete_state_transition,
                            EDGE_ORDER, EDGE_SPACING, ANIMALS)

from sklearn.model_selection import KFold
from tqdm.auto import tqdm

from src.load_data import make_track_graph
import numpy as np


track_graph, center_well_id = make_track_graph(epoch_key, ANIMALS)
is_running = np.abs(data["position_info"].tailBase_vel) > 4
is_outbound = data["position_info"].task == "Outbound"

classifier_parameters = {
    'movement_var': 6.0,
    'replay_speed': 1,
    'place_bin_size': 2.5,
    'continuous_transition_types': [['random_walk', 'uniform'],
                                    ['uniform',     'uniform']],
    'knot_spacing': 5,
    'spike_model_penalty': 0.5
}

cv = KFold()
cv_classifier_sorted_spikes_results = []

for fold_ind, (train, test) in tqdm(enumerate(cv.split(data["position_info"].index))):
    
    #train = train[is_outbound[train].values]
    
    cv_classifier = SortedSpikesClassifier(**classifier_parameters)

    cv_classifier.fit(
        position=data["position_info"].iloc[train].linear_position,
        spikes=data["spikes"].iloc[train],
        is_training=is_running.iloc[train],
        track_graph=track_graph,
        center_well_id=center_well_id,
        edge_order=EDGE_ORDER,
        edge_spacing=EDGE_SPACING,
    )
    cv_classifier.discrete_state_transition_ = discrete_state_transition
    logging.info('Predicting posterior...')
    cv_classifier_sorted_spikes_results.append(
        cv_classifier.predict(
            data["spikes"].iloc[test],
            time=data["position_info"].iloc[test].index / np.timedelta64(1, "s"),
        )
    )
    
# concatenate cv classifier results 
cv_classifier_sorted_spikes_results = xr.concat(
    cv_classifier_clusterless_results, dim="time"
)
cv_classifier_sorted_spikes_results

In [ ]:
# save the results as .nc format. ncread matlab can read these
cv_classifier_clusterless_results.to_netcdf(
   f"{epoch_key[0]}_{epoch_key[1]:02d}_{epoch_key[2]:02d}_cv_classifier_sortedspikes_vel_4_nose_alltime_results.nc"
)